Подключим необходимые библиотеки

In [159]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import re 
from scipy.sparse import hstack as csr_hstack
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import zipfile
#from zipfile_infolist import print_info

import math
from sklearn.cross_validation import KFold
import xgboost as xgb

import os
import numpy as np
import pandas as pd

import time

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

Populating the interactive namespace from numpy and matplotlib


Считаем файлы с трейном и тестом

In [160]:
%%time
dtrain = pd.read_csv('Xtrain.csv')#.fillna("")
dtest = pd.read_csv('Xtest.csv')#.fillna("")
print np.shape(dtrain)
print np.shape(dtest)

(300000, 25)
(100000, 16)
CPU times: user 2.92 s, sys: 140 ms, total: 3.06 s
Wall time: 3.73 s


In [161]:
#dtrain.head(20)

Объединим их в одну таблицу

In [162]:
%%time
dall = pd.concat((dtrain, dtest), axis = 0)
dall.set_index(np.arange(len(dall)), inplace = True)

dall = dall[dtrain.columns]

CPU times: user 488 ms, sys: 88 ms, total: 576 ms
Wall time: 570 ms


Прочитаем столбцы которые нам нужно предсказать, заменим внутри каждого столбца класс на номер класса и запомним названия в формате задачи

In [163]:
#Типы полей по условиям задачи:
target_columns=['Function','Use','Sharing','Reporting','Student_Type','Position_Type','Object_Type','Pre_K','Operating_Status']
cont_columns=['FTE','Total']
other_columns=['Object_Description','Text_2','SubFund_Description','Job_Title_Description','Text_3','Text_4','Sub_Object_Description','Location_Description','Function_Description','Facility_or_Department','Position_Extra','Program_Description','Fund_Description','Text_1']

In [164]:
maps = []
colnames = []

for colpred in dall.columns[:9]:
    classes = sort(dtrain[colpred].unique())
    
    maps.append(dict(zip(classes, np.arange(len(classes)))))
    
    for i in range(len(classes)):
        colnames.append(colpred + '__' + classes[i])
    
    dall[colpred].replace(maps[-1], inplace = True)

# Все текстовые признаки переведем в числовые
## with category counting

In [18]:
%%time
# по частоте
for col in other_columns: #dall.columns[9:]:
    dall[col]=dall[col].fillna("")
    dall[col] = dall.groupby(col)[col].transform(len)
#dall['Object_Description'] = dall.groupby('Object_Description')['Object_Description'].transform(len)

CPU times: user 3.94 s, sys: 0 ns, total: 3.94 s
Wall time: 3.93 s


In [121]:
%%time
# просто кодировкой
for col in other_columns:#dall.columns[9:]:
    dall[col]=dall[col].fillna("")
    dall[col] = LabelEncoder().fit_transform(dall[col])

CPU times: user 6.08 s, sys: 32 ms, total: 6.11 s
Wall time: 6.1 s


In [82]:
#dall=pd.DataFrame(dall)
dall[dall.columns[9:]].head(3)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
0,495,0,135,0,0,0,0,7,20992,340,0,0,200086,226,51,0
1,502,0,7,2950,0,0,51,319,20992,312,84,106,83560,4,4,933
2,467,272,95,3367,25,0,0,0,640,664,0,422,58463,135,0,1097


In [50]:
for col in dtrain[target_columns].columns:
    print len(unique(dtrain[col]))

37
8
5
3
9
25
11
3
3


# TEXT, TF-IDF

In [172]:
dall[other_columns].head(2)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,Function_Description,Facility_or_Department,Position_Extra,Program_Description,Fund_Description,Text_1
0,SUPPLIES,,MISCELLANEOUS,,,,,ADMIN. SERVICES,MAINTENANCE AND OPERATING,,,Misc,General Fund,
1,Salaries And Wages For Substitute Professionals,,"""Title Part A Improving Basic Programs""",TEACHER SUBSTITUTE POOL,,,Extra Substitute Teachers Grant Funded,School,Instruction,Instruction And Curriculum,CERTIFIED SUBSTITUTE,"""Title I, Part A Schoolwide Activities Related...","""Title Part A Improving Basic Programs""",MISCELLANEOUS


In [165]:
tfidf = TfidfVectorizer(ngram_range=(1, 2))

st_other_columns=[]
for col in other_columns:
    print col   
    dall[col] = dall[col].fillna("")
    if col=='Object_Description':
        XallIdf = tfidf.fit_transform(dall[col]) 
    else:
        XallIdf = csr_hstack([XallIdf,tfidf.fit_transform(dall[col])])
    print np.shape(XallIdf)[1]

Object_Description
1489
Text_2
2213
SubFund_Description
3133
Job_Title_Description
9210
Text_3
9321
Text_4
10079
Sub_Object_Description
10686
Location_Description
11644
Function_Description
13503
Facility_or_Department
13961
Position_Extra
15081
Program_Description
16351
Fund_Description
16872
Text_1
20632


In [166]:
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components = 50)
XIdf2sdv50 = pca.fit_transform(XallIdf)
print type(XIdf2sdv50)
print shape(XIdf2sdv50)

<type 'numpy.ndarray'>
(400000, 50)


In [44]:
type(XallIdf)

scipy.sparse.coo.coo_matrix

In [118]:
Xall=hstack([Xother_encoded,XIdf2sdv10])
print np.shape(Xall)

(400000, 24)


In [38]:
#dall.drop('merge', axis=1, inplace=True)
dall[dall.columns[9:]].head(2)

,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1,merge
0,SUPPLIES,,MISCELLANEOUS,,,,,ADMIN. SERVICES,NaN,MAINTENANCE AND OPERATING,,,6179.91,Misc,General Fund,,SUPPLIES MISCELLANEOUS ADMIN. SERVICES MA...
1,Salaries And Wages For Substitute Professionals,,"""Title Part A Improving Basic Programs""",TEACHER SUBSTITUTE POOL,,,Extra Substitute Teachers Grant Funded,School,NaN,Instruction,Instruction And Curriculum,CERTIFIED SUBSTITUTE,275.93,"""Title I, Part A Schoolwide Activities Related...","""Title Part A Improving Basic Programs""",MISCELLANEOUS,Salaries And Wages For Substitute Professional...


## Count Vectorizer

In [174]:
dall['merge'] = ''

for col in other_columns: #dall.columns[9:]:
    dall['merge'] += dall[col].astype(str) + ' '

In [175]:
%%time
#XallCV5 = CountVectorizer().fit_transform(dall.ix[:, -1])
XallCV = CountVectorizer(min_df = 5).fit_transform(dall.ix[:, -1])
print np.shape(XallCV)
print type(XallCV)

(400000, 2676)
<class 'scipy.sparse.csr.csr_matrix'>
CPU times: user 18.7 s, sys: 116 ms, total: 18.8 s
Wall time: 18.8 s


In [176]:
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components = 20)
Xall = pca.fit_transform(XallCV)
print shape(Xall)
print np.type(Xall)

(400000, 20)


AttributeError: 'module' object has no attribute 'type'

In [177]:
#Xall=hstack([Xother_encoded,Xcv5svd100])
#Xall=XallIdf.tocsr()
print type(Xall)

<type 'numpy.ndarray'>


# FTE & Total

In [94]:
#dall['Totalq100'] = pd.qcut(dall['Total'], 100, labels=False)
#dall['Totalq50'] = pd.qcut(dall['Total'], 50, labels=False)
#dall['Totalq1000'] = pd.qcut(dall['Total'], 1000, labels=False)
dall['Total100'] = pd.cut(dall['Total'], 100, labels=False)
dall['Total50'] = pd.cut(dall['Total'], 50, labels=False)
dall['Total1000'] = pd.cut(dall['Total'], 1000, labels=False)

dall['FTE100'] = pd.cut(dall['FTE'], 100, labels=False)
dall['FTE50'] = pd.cut(dall['FTE'], 50, labels=False)
dall['FTE1000'] = pd.cut(dall['FTE'], 1000, labels=False)

q_cols=['FTE100', 'FTE50', 'FTE1000','Total100','Total50', 'Total1000']
dall[q_cols].fillna(-1, inplace = True)

/home/atselikov/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [90]:
sum(dall['Total100'].isnull())

#dall['Total100'].fillna(-1, inplace = True)
#dall[q_cols].describe(include='all')
#q_cols+other_columns

0

In [95]:
# FTE && Total

# NA & zero
dall['FteZero'] = (dall['FTE'] == 0.0).astype(int)
dall['TotalZero'] = (dall['Total'] == 0.0).astype(int)

dall['NaFTE'] = dall['FTE'].isnull().astype(int)
dall['NaTotal'] = dall['Total'].isnull().astype(int)

dall.loc[dall["FTE"].isnull(), "FTE"] = 0
dall.loc[dall["Total"].isnull(), "Total"] = 0

# отрицательные
dall['FteIsNeg'] = (dall['FTE']<0).astype(int)
dall.loc[dall["FTE"] < 0, "FTE"] = 0

# большие
dall['Fte5'] = (dall['FTE']>5).astype(int)
dall.loc[dall["FTE"] > 5, "FTE"] = 5                                    

# total neg
dall['TotalIsNeg'] = (dall['Total']<0).astype(int)
dall['TotalNeg'] = dall['Total'][dall['Total']<0]
dall["TotalNeg"][dall['TotalNeg'].isnull()] = 0
dall['TotalNegLog'] = log(1-dall['TotalNeg'])

dall['TotalPos'] = dall['Total'][dall['Total']>0]
dall["TotalPos"][dall['TotalPos'].isnull()] = 0
dall['TotalPosLog'] = log(1+dall['TotalPos'])

fte_cols=['FTE','NaFTE', 'FteZero', 'FteIsNeg','Fte5']exit

#total_cols=['Total','NaTotal', 'TotalZero', 'TotalIsNeg','TotalNeg', 'TotalNegLog', 'TotalPos', 'TotalPosLog']
total_cols=['NaTotal', 'TotalZero', 'TotalIsNeg', 'TotalNegLog', 'TotalPosLog']
dall.drop(['TotalNeg', 'TotalPos', 'Total'], axis=1, inplace=True)

/home/atselikov/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/atselikov/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [109]:
dall[q_cols]=dall[q_cols].fillna(-1)
total_cols=['NaTotal', 'TotalZero', 'TotalIsNeg', 'TotalNegLog', 'TotalPosLog']
dall[total_cols+fte_cols+q_cols].head(3)

,NaTotal,TotalZero,TotalIsNeg,TotalNegLog,TotalPosLog,FTE,NaFTE,FteZero,FteIsNeg,Fte5,FTE100,FTE50,FTE1000,Total100,Total50,Total1000
0,0,0,0,0,8.729221,0.00000,1,0,0,0,-1,-1,-1,40,20,402
1,0,0,0,0,5.623765,0.00000,1,0,0,0,-1,-1,-1,40,20,402
2,0,0,0,0,4.704273,0.00431,0,0,0,0,0,0,1,40,20,402


In [110]:
sum(dall[q_cols].isnull())

FTE100       0
FTE50        0
FTE1000      0
Total100     0
Total50      0
Total1000    0
dtype: int64

# Words statistics

In [149]:
%%time
from __future__ import division
words_stst_cols=['NanCount']
dall['NanCount'] = dall[other_columns].isnull().sum(axis=1)

for col in other_columns:    
    dall[col + 'WCount'] = dall[col].map(lambda x: len(str(x).split())).astype(int)    #words count
    dall[col + 'SymCount'] = dall[col].map(lambda x: len(str(x))).astype(int)  #symb count    
    dall[col + 'WSRatio'] = dall[col + 'WCount'] / dall[col + 'SymCount'] #words-symbs ratio
    dall[col + 'IsNa'] = dall[col].isnull().astype(int)
    words_stst_cols.append(col + 'WCount')
    words_stst_cols.append(col + 'SymCount')
    words_stst_cols.append(col + 'WSRatio')
    words_stst_cols.append(col + 'IsNa')    

CPU times: user 13 s, sys: 60 ms, total: 13 s
Wall time: 12.9 s


In [41]:
print len(words_stst_cols)
dall[words_stst_cols].describe(include='all')

57


,NanCount,Object_DescriptionWCount,Object_DescriptionSymCount,Object_DescriptionWSRatio,Object_DescriptionIsNa,Text_2WCount,Text_2SymCount,Text_2WSRatio,Text_2IsNa,SubFund_DescriptionWCount,SubFund_DescriptionSymCount,SubFund_DescriptionWSRatio,SubFund_DescriptionIsNa,Job_Title_DescriptionWCount,Job_Title_DescriptionSymCount,Job_Title_DescriptionWSRatio,Job_Title_DescriptionIsNa,Text_3WCount,Text_3SymCount,Text_3WSRatio,Text_3IsNa,Text_4WCount,Text_4SymCount,Text_4WSRatio,Text_4IsNa,...,Location_DescriptionIsNa,Function_DescriptionWCount,Function_DescriptionSymCount,Function_DescriptionWSRatio,Function_DescriptionIsNa,Facility_or_DepartmentWCount,Facility_or_DepartmentSymCount,Facility_or_DepartmentWSRatio,Facility_or_DepartmentIsNa,Position_ExtraWCount,Position_ExtraSymCount,Position_ExtraWSRatio,Position_ExtraIsNa,Program_DescriptionWCount,Program_DescriptionSymCount,Program_DescriptionWSRatio,Program_DescriptionIsNa,Fund_DescriptionWCount,Fund_DescriptionSymCount,Fund_DescriptionWSRatio,Fund_DescriptionIsNa,Text_1WCount,Text_1SymCount,Text_1WSRatio,Text_1IsNa
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.00000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,...,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000
mean,6.476895,3.319678,25.347213,0.142324,0.061910,1.239993,5.995737,0.288188,0.779617,2.209565,13.761680,0.195544,0.233395,2.266128,16.85772,0.182528,0.268632,1.003245,4.227952,0.279845,0.550390,1.286615,5.934155,0.305756,0.865745,...,0.595150,2.283202,17.954630,0.151193,0.145095,1.223413,5.487578,0.305749,0.865398,1.408573,14.254215,0.171071,0.338558,2.526430,19.188198,0.175863,0.238885,2.086368,14.386278,0.231052,0.493175,1.932750,14.563210,0.181071,0.269775
std,2.280374,1.875631,12.597680,0.058401,0.240992,0.623327,6.413024,0.086432,0.414505,1.372431,9.668098,0.081801,0.422992,1.165351,10.06648,0.096186,0.443249,0.074562,2.166004,0.087710,0.497455,0.969169,8.816446,0.071717,0.340927,...,0.490864,1.561688,11.360369,0.087100,0.352197,0.605430,6.582186,0.070932,0.341299,0.900649,9.734214,0.123401,0.473220,1.771927,13.301738,0.094793,0.426403,1.552680,14.356981,0.104493,0.499954,0.956156,8.641801,0.098051,0.443843
min,2.000000,1.000000,3.000000,0.033333,0.000000,1.000000,2.000000,0.035714,0.000000,1.000000,3.000000,0.071429,0.000000,0.000000,3.00000,0.000000,0.000000,1.000000,3.000000,0.071429,0.000000,1.000000,3.000000,0.071429,0.000000,...,0.000000,1.000000,3.000000,0.033333,0.000000,1.000000,3.000000,0.033333,0.000000,1.000000,3.000000,0.025000,0.000000,1.000000,2.000000,0.033333,0.000000,1.000000,3.000000,0.038462,0.000000,0.000000,1.000000,0.000000,0.000000
25%,5.000000,2.000000,17.000000,0.117647,0.000000,1.000000,3.000000,0.333333,1.000000,1.000000,5.000000,0.133333,0.000000,1.000000,3.00000,0.111111,0.000000,1.000000,3.000000,0.142857,0.000000,1.000000,3.000000,0.333333,1.000000,...,0.000000,1.000000,11.000000,0.090909,0.000000,1.000000,3.000000,0.333333,1.000000,1.000000,3.000000,0.062500,0.000000,1.000000,4.000000,0.107143,0.000000,1.000000,3.000000,0.136364,0.000000,1.000000,3.000000,0.105263,0.000000
50%,6.000000,3.000000,28.000000,0.133333,0.000000,1.000000,3.000000,0.333333,1.000000,2.000000,12.000000,0.166667,0.000000,2.000000,20.00000,0.142857,0.000000,1.000000,3.000000,0.333333,1.000000,1.000000,3.000000,0.333333,1.000000,...,1.000000,2.000000,14.000000,0.108696,0.000000,1.000000,3.000000,0.333333,1.000000,1.000000,12.000000,0.125000,0.000000,2.000000,20.00

# Prepare to RUN

In [181]:
best3ws_cols = ['Job_Title_DescriptionWSRatio','Job_Title_DescriptionSymCount','Position_ExtraWSRatio']
best6ws_cols=['Job_Title_DescriptionWSRatio','Job_Title_DescriptionSymCount','Object_DescriptionWSRatio','Function_DescriptionWSRatio','Object_DescriptionSymCount','Text_1WSRatio']
BoWcols = ['X19','X0','X16','X7','X17','X15','X9']
BoWcolsNums = [19,0,16,7,17,15,9]
best2FteTot = ['TotalPosLog','TotalNegLog','FTE']
tot_fte_cols=total_cols+fte_cols+q_cols
type(tot_fte_cols)

list

In [184]:
np.shape(Xall[:,BoWcolsNums])

(400000, 7)

In [201]:
#Xall = np.array(dall.ix[:, 9:])
#Xall = np.array(dall[tot_fte_cols])
#Xall = np.array(dall[other_columns+best6ws_cols])
#words_stst_cols

####    GOLD!!!!!! Xall=np.array(dall[other_columns+best2FteTot])
#XotherBest2FteTot=np.copy(Xall)
#XothBst2FtT_3ws = np.copy(Xall)

#Xall = np.hstack((XotherBest2FteTot,np.array(dall[best3ws_cols])))

Xall=np.copy(XothBst2FtT_3ws)

#Xall = np.hstack((XothBst2FtT_3ws,Xall[:,BoWcolsNums]))

#Xall=np.array(dall[other_columns+best2FteTot+best3ws_cols])

#all_col=other_columns+best2FteTot+best3ws_cols+BoWcols

#Xother=np.copy(Xall)
#Xother_best6ws=np.copy(Xall)
#Xall = np.array(dall[other_columns+best4ws_cols])#+q_cols])

In [151]:
dall[best3ws_cols].head(2)

,Job_Title_DescriptionWSRatio,Job_Title_DescriptionSymCount,Position_ExtraWSRatio
0,0.333333,3,0.333333
1,0.130435,23,0.100000


In [202]:
pd.DataFrame(Xall).head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,495,0,135,0,0,0,0,7,340,0,0,226,51,0,8.729221,0,0,0.333333,3,0.333333
1,502,0,7,2950,0,0,51,319,312,84,106,4,4,933,5.623765,0,0,0.130435,23,0.100000


In [195]:
final_list=['TotalPosLog','Job_Title_Description','Object_Description','Function_Description','Location_Description','Text_1','TotalNegLog','Program_Description','Job_Title_DescriptionWSRatio','SubFund_Description','Job_Title_DescriptionSymCount','Position_Extra','Text_2','Fund_Description','Sub_Object_Description']

In [196]:
np.shape(XallFinal[final_list])

(400000, 15)

In [197]:
Xall = np.array(XallFinal[final_list])
np.shape(Xall)

(400000, 15)

In [187]:
gibaindtrain1 = []
gibaindtrain2 = []
CFOLDS=5

for giba in range(CFOLDS):
    gibaindtrain1.append(arange(len(dtrain))[arange(len(dtrain)) % CFOLDS != giba])
    gibaindtrain2.append(arange(len(dtrain))[arange(len(dtrain)) % CFOLDS == giba])

# Feature Importance

In [43]:
    f_columns = dall[words_stst_cols].columns
    for colpred in dall.columns[:1]:
            indtrain1 = gibaindtrain1[0]
            Xtrain = Xall[:len(dtrain)]
            #Xtest = Xall[len(dtrain):]
            ytrain = np.array(dall[colpred])[:len(dtrain)]
            
            Xtrain = pd.DataFrame(Xtrain).loc[indtrain1]
            ytrain = ytrain[indtrain1]
            
            model = RandomForestClassifier(n_estimators = 200, n_jobs = -1, random_state=rs)
            model.fit(Xtrain, ytrain)
            feat_imp = pd.DataFrame(model.feature_importances_, index=f_columns,
             columns=['Importance']).sort(['Importance'], ascending=False)            

/home/atselikov/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [190]:
%%time
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

param = {}
param['booster'] = 'gbtree'
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'rmse'
param['num_class'] = 37
param['bst:eta'] = 0.4
param['nthread'] = 15
param['silent'] = 1
param['bst:max_depth'] = 15
param['bst:colsample_bytree'] = 0.85
num_round = 50
watchlist = []
col_set=dall.columns[:1]
rs=10
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 37))

col_names=[]
#f_columns = dall[tot_fte_cols].columns
#create_feature_map(f_columns)
for colpred in col_set:
        indtrain1 = gibaindtrain1[0]
        Xtrain = Xall[:len(dtrain)]
        ytrain = np.array(dall[colpred])[:len(dtrain)]        
        Xtrain = pd.DataFrame(Xtrain).loc[indtrain1]
        ytrain = ytrain[indtrain1]
        for i in range(len(Xtrain.columns)):
            col_names.append('X'+str(i))
        Xtrain.columns =col_names
        mod = xgb.train(param, xgb.DMatrix(data = Xtrain, label = ytrain), num_round, watchlist)

CPU times: user 1h 1min 27s, sys: 12 s, total: 1h 1min 39s
Wall time: 5min 10s


In [191]:
import operator
create_feature_map(all_col)
importance = mod.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

In [193]:
df

,feature,fscore
0,TotalPosLog,0.215775
1,Job_Title_Description,0.088534
2,Object_Description,0.066864
3,Function_Description,0.065779
4,Location_Description,0.064962
5,Text_1,0.062799
6,TotalNegLog,0.051254
7,Program_Description,0.042032
8,Job_Title_DescriptionWSRatio,0.041619
9,SubFund_Description,0.033910


In [44]:
df.head(20)

,Importance
Job_Title_DescriptionWSRatio,0.072164
Job_Title_DescriptionSymCount,0.053901
Position_ExtraWSRatio,0.047599
Job_Title_DescriptionWCount,0.042202
Function_DescriptionWSRatio,0.038622
Position_ExtraSymCount,0.036254
Object_DescriptionWSRatio,0.035929
Location_DescriptionWSRatio,0.034856
Object_DescriptionSymCount,0.034322
Program_DescriptionWSRatio,0.033289


# RUN VALIDATION

In [49]:
def my_rmse(yt, yp):    
    return sqrt(mean_squared_error(yt, yp))

def fold_score(CVXtrain, CVXtest, CVytrain, CVytest, col_set, model):
    yallpred = np.array([]).reshape(len(CVXtest), 0)
    yalltest = np.array([]).reshape(len(CVXtest), 0)
    for colpred in col_set:
        #print colpred
        model.fit(CVXtrain, CVytrain[colpred])    
        ypred = model.predict_proba(CVXtest)
        yallpred = np.hstack((yallpred, ypred))
        yalltest = np.hstack((yalltest, pd.get_dummies(CVytest[colpred]))) 
    print my_rmse(yalltest, yallpred)        
    return my_rmse(yalltest, yallpred), yallpred

def fold_score_csr(CVXtrain, CVXtest, CVytrain, CVytest, col_set, model):
    yallpred = np.array([]).reshape(shape(CVXtest)[0], 0)
    yalltest = np.array([]).reshape(shape(CVXtest)[0], 0)
    for colpred in col_set:
        model.fit(CVXtrain, np.array(CVytrain[colpred]))
        ypred = model.predict_proba(CVXtest)
        yallpred = np.hstack((yallpred, ypred))
        yalltest = np.hstack((yalltest, pd.get_dummies(CVytest[colpred])))  
    print my_rmse(yalltest, yallpred) 
    return my_rmse(yalltest, yallpred), yallpred
   
def get_kfold_scores(X, y, clf, random_state):
    score_total = 0.0
    for giba in range(CFOLDS):
        #print giba
        indtrain1 = gibaindtrain1[giba]
        indtrain2 = gibaindtrain2[giba]
        
        X_train = pd.DataFrame(X).loc[indtrain1]
        y_train = y.loc[indtrain1]
        X_test = pd.DataFrame(X).loc[indtrain2]
        y_test = y.loc[indtrain2]
        
        #curres, ypred = stasknn(indtrain1, indtrain2)
        
        score, foldpred =fold_score(X_train, X_test, y_train, y_test, col_set, clf)

        arres.append(score)
        #arrlos.append(losses)    
        allypred[indtrain2, :] = foldpred
        
        score_total += score
        
    print np.mean(arres, axis = 0)
    average_score = score_total/float(CFOLDS)
    print str(average_score)

def get_kfold_scores_csr(X, y, clf, random_state):
    score_total = 0.0
    for giba in range(CFOLDS):
        indtrain1 = gibaindtrain1[giba]
        indtrain2 = gibaindtrain2[giba]
        
        X_train = X[indtrain1]
        y_train = y.loc[indtrain1]
    	X_test = X[indtrain2]
    	y_test = y.loc[indtrain2]
        
        score, foldpred =fold_score_csr(X_train, X_test, y_train, y_test, col_set, clf)

        arres.append(score)
        #arrlos.append(losses)    
        allypred[indtrain2, :] = foldpred
        
        score_total += score
        
    print np.mean(arres, axis = 0)
    average_score = score_total/float(CFOLDS)
    print average_score

# SVM proba

In [71]:
from sklearn.linear_model import SGDClassifier
col_set=dall.columns[:9]
rs=10
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 104))
clf = SGDClassifier(loss = 'log', n_jobs=15)
get_kfold_scores_csr(Xall[:len(dtrain)], dall[col_set][:len(dtrain)], clf, random_state  = rs)

0.0723647226487
0.0727915716928
0.0713942836447
0.0714060886057
0.0709638209555
0.0717840975095
0.0717840975095


## create train-test CSR

In [72]:
m1train=np.copy(allypred)
np.shape(allypred)

In [73]:
yallpred = np.array([]).reshape(len(dtest), 0)

for colpred in dall.columns[:9]:
    Xtrain = Xall[:len(dtrain)]
    Xtest = Xall[len(dtrain):]
    
    ytrain = np.array(dall[colpred])[:len(dtrain)]
    
    model = SGDClassifier(loss = 'log', n_jobs=15)
    model.fit(Xtrain, ytrain)
    
    ypred = model.predict_proba(Xtest)
    
    yallpred = np.hstack((yallpred, ypred))
    
m1test=np.copy(yallpred)
np.shape(m1test)

(300000, 104)

In [74]:
m1test=np.copy(yallpred)
np.shape(m1test)

(100000, 104)

# RF  By all columns (or by subset)

In [126]:
%%time
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
col_set=dall.columns[:1]
rs=10
#folds=3
est_count=50
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 37))
clf = RandomForestClassifier(n_estimators = est_count, n_jobs = 15, random_state=rs) #0.098330060692
get_kfold_scores(Xall[:len(dtrain)], dall[col_set][:len(dtrain)], clf, random_state  = rs)

0.04538322218
0.045643706343
0.0456892573063
0.0453490939819
0.0453475649932
0.0454825689609
0.0454825689609
CPU times: user 9min 58s, sys: 14.1 s, total: 10min 12s
Wall time: 48.6 s


In [114]:
m3train = np.copy(allypred)
np.shape(m3train)

(300000, 104)

In [50]:
yallpred = np.array([]).reshape(len(dtest), 0)
for colpred in dall.columns[:9]:
    Xtrain = Xall[:len(dtrain)]
    Xtest = Xall[len(dtrain):]
    ytrain = np.array(dall[colpred])[:len(dtrain)]
    model = AdaBoostClassifier(base_estimator = ExtraTreesClassifier(n_estimators = est_count, criterion = 'entropy', n_jobs = -1),
                             n_estimators = 3)            
            #model = ExtraTreesClassifier(n_estimators = est_count, n_jobs = -1, random_state=rs)
    model.fit(Xtrain, ytrain)
    ypred = model.predict_proba(Xtest)
    yallpred = np.hstack((yallpred, ypred))
    print colpred

KeyboardInterrupt: 

In [19]:
def rf_predict(est_count):
    t0 = time.time()
    yallpred = np.array([]).reshape(len(dtest), 0)
    #yfinalet = np.array([]).reshape(len(dtest), 0)
    for colpred in dall.columns[:9]:
            Xtrain = Xall[:len(dtrain)]
            Xtest = Xall[len(dtrain):]
            ytrain = np.array(dall[colpred])[:len(dtrain)]
            #model = RandomForestClassifier(n_estimators = est_count, n_jobs = -1, random_state=rs)
            model = RandomForestClassifier(n_estimators = est_count, n_jobs = -1, random_state=rs)
            for bg in range(3):    
                model = AdaBoostClassifier(base_estimator = ExtraTreesClassifier(n_estimators = est_count, criterion = 'entropy', n_jobs = -1),
                             n_estimators = 3, random_state = bg + 1)            
                model.fit(Xtrain, ytrain)
                ypred = model.predict_proba(Xtest)
                yfinalet += ypred
                print bg, (time.time() - t0) / 60
            yfinalet /= bgs                
            yallpred = np.hstack((yallpred, yfinalet))
            print colpred, (time.time() - t0) / 60. ##0.83 
    return yallpred

In [ ]:
rs=10
yallpred=rf_predict(200)

In [117]:
m3test = np.copy(yallpred)
np.shape(m3test)

(100000, 104)

#    ========================== XGB =================

In [129]:
def fold_score_xgb(CVXtrain, CVXtest, CVytrain, CVytest, col_set):
    yallpred = np.array([]).reshape(len(CVXtest), 0)
    yalltest = np.array([]).reshape(len(CVXtest), 0)
    col_names=[]
    for i in range(len(CVXtrain.columns)):
        col_names.append('X'+str(i))
    CVXtrain.columns =col_names
    CVXtest.columns =col_names
    i=0
    classes = [37,8,5,3,9,25,11,3,3]
    for colpred in col_set:
        print colpred
        param['num_class'] = classes[i]
        mod = xgb.train(param, xgb.DMatrix(data = CVXtrain, label = CVytrain[colpred]), num_round, watchlist)        
        ypred = mod.predict(xgb.DMatrix(CVXtest))
        yallpred = np.hstack((yallpred, ypred))
        
        yalltest = np.hstack((yalltest, pd.get_dummies(CVytest[colpred])))  
        i +=1
    print my_rmse(yalltest, yallpred)
    return my_rmse(yalltest, yallpred), yallpred

	
def get_kfold_scores_xgb(X, y, random_state):
    score_total = 0.0
    for giba in range(CFOLDS):
        if giba>0:
            print np.mean(arres, axis = 0)
            return
        indtrain1 = gibaindtrain1[giba]
        indtrain2 = gibaindtrain2[giba]
        X_train = pd.DataFrame(X).loc[indtrain1]
        y_train = y.loc[indtrain1]
        X_test = pd.DataFrame(X).loc[indtrain2]
        y_test = y.loc[indtrain2]
        score, foldpred =fold_score_xgb(X_train, X_test, y_train, y_test, col_set)
        arres.append(score)
        allypred[indtrain2, :] = foldpred
        score_total += score
    print np.mean(arres, axis = 0)
    average_score = score_total/float(CFOLDS)
    print str(average_score)


# CV XGB BY Class1 fold1

In [203]:
%%time
param = {}
param['booster'] = 'gbtree'
#param['booster'] = 'gblinear'
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'rmse'
#param['num_class'] = 37
param['bst:eta'] = 0.13
param['nthread'] = 15
param['silent'] = 1
param['bst:max_depth'] = 15
param['bst:colsample_bytree'] = 0.85
#param['gamma'] = 0.5

num_round = 150
watchlist = []

col_set=dall.columns[:1]
rs=10
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 37))
get_kfold_scores_xgb(Xall[:len(dtrain)], dall[col_set][:len(dtrain)], random_state  = rs)

Function
0.0366251021844
0.0366251021844
CPU times: user 3h 34min 58s, sys: 38.8 s, total: 3h 35min 37s
Wall time: 19min 47s


# CV XGB BY ALL CLASSES of FOLD1

In [157]:
%%time
param = {}
param['booster'] = 'gbtree'
#param['booster'] = 'gblinear'
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'rmse'
#param['num_class'] = 37
param['bst:eta'] = 0.4
param['nthread'] = 15
param['silent'] = 1
param['bst:max_depth'] = 15
param['bst:colsample_bytree'] = 0.85
#param['gamma'] = 0.5

num_round = 50
watchlist = []

col_set=dall.columns[:9]
rs=10
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 104))
get_kfold_scores_xgb(Xall[:len(dtrain)], dall[col_set][:len(dtrain)], random_state  = rs)

Function
Use
Sharing
Reporting
Student_Type
Position_Type
Object_Type
Pre_K
Operating_Status
0.0452617627268
0.0452617627268
CPU times: user 3h 50min 59s, sys: 54 s, total: 3h 51min 53s
Wall time: 22min 13s


### xgb predict

In [205]:
print num_round
print param

150
{'bst:colsample_bytree': 0.85, 'num_class': 37, 'silent': 1, 'eval_metric': 'rmse', 'nthread': 15, 'bst:max_depth': 15, 'objective': 'multi:softprob', 'bst:eta': 0.13, 'booster': 'gbtree'}


In [206]:
t0 = time.time()
yallpred = np.array([]).reshape(len(dtest), 0)
i=0
classes = [37,8,5,3,9,25,11,3,3]
for colpred in dall.columns[:9]:
    Xtrain = Xall[:len(dtrain),]
    Xtest = Xall[len(dtrain):,]    
    ytrain = np.array(dall[colpred])[:len(dtrain)]    
    param['num_class'] = classes[i]
    mod = xgb.train(param, xgb.DMatrix(data = Xtrain, label = ytrain), num_round, watchlist)        
    ypred = mod.predict(xgb.DMatrix(Xtest))
    yallpred = np.hstack((yallpred, ypred))
    i +=1
    print colpred, (time.time() - t0) / 60

Function 30.8445929488
Use 38.6506843646
Sharing 42.922522366
Reporting 45.501677235
Student_Type 51.6783275485
Position_Type 66.4205199162
Object_Type 74.0823821823
Pre_K 76.5909155647
Operating_Status 79.5430191318


In [207]:
np.shape(yallpred)

(100000, 104)

In [208]:
file_name = 'xbgOth2FteTot3Num150'
submit(yallpred, colnames, file_name)

submitted!


### By target columns by order

In [18]:
from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=rs)
#clfbag = BaggingClassifier(clf, n_estimators=50, random_state=rs)
#calibrated_clf = CalibratedClassifierCV(clf, method='isotonic', cv=5)

In [77]:
last_imp #=['Text_4', 'Text_4', 'Text_4', 'Text_4', 'Facility_or_Department', 'Facility_or_Department', 'Facility_or_Department', 'Text_3', 'Text_4']
#'Facility_or_Department','Facility_or_Department','Facility_or_Department','Facility_or_Department','Sub_Object_Description','Text_4','Text_4','Text_2','Facility_or_Department'
last_impn =[5,5,5,5,9,9,9,4,5]
last_impn2 =[9,9,9,9,6,5,5,1,9]
last_impn3 =[4,6,0,10,4,6,1,13,4]

all1=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [60]:
#pd.DataFrame(Xall[:,all1])
pd.DataFrame(Xall[:len(dtrain)])
Xall1 = Xall[:len(dtrain)]
#pd.DataFrame(Xall[:,all1])

In [52]:
%%time
rs=10
#folds=3
est_count=200
tc_len=[37,8,5,3,9,25,11,3,3]
Xall1 = Xall[:len(dtrain)]
clf = RandomForestClassifier(n_estimators = est_count, n_jobs = 15, random_state=rs)
for j in np.arange(1, 10, 1):
    allypred = np.zeros((len(dtrain), tc_len[j-1]))
    col_set=dall.columns[j-1:j]
    all1=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    #all1.remove(last_impn[j-1])
    #all1.remove(last_impn2[j-1])
    #all1.remove(last_impn3[j-1])
    get_kfold_scores(Xall1[:,all1], dall[col_set][:len(dtrain)], clf, random_state  = rs)

0
1
2
3
4
0.0423149114687
0.0423005564102
0
1
2
3
4
0.0565958557328
0.0765891777025
0
1
2
3
4
0.0656449862881
0.087362899621
0
1
2
3
4
0.073073412289
0.098330060692
0
1
2
3
4
0.0708691439918
0.0611703634842
0
1
2
3
4
0.0654671328144
0.036296272456
0
1
2
3
4
0.0635095574068
0.0509810747987
0
1
2
3
4
0.0648441931039
0.0747204972619
0
1
2
3
4
0.0672530786873
0.0874877175882
CPU times: user 2h 19min 20s, sys: 2min 4s, total: 2h 21min 24s
Wall time: 10min 50s


# CREATE SUBMISSION

In [58]:
def rf_predict_imp(rs, est_count):
    t0 = time.time()
    yallpred = np.array([]).reshape(len(dtest), 0)
    
    for colpred in dall.columns[:9]:
            Xtrain = Xall[:len(dtrain)]
            Xtest = Xall[len(dtrain):]
            ytrain = np.array(dall[colpred])[:len(dtrain)]
            model = RandomForestClassifier(n_estimators = est_count, n_jobs = -1, random_state=rs)
            model.fit(Xtrain, ytrain)
            feat_imp = pd.DataFrame(model.feature_importances_,
             index=dall[other_columns].columns,
             columns=['Importance']).sort(['Importance'], ascending=False)
            feat_imp.to_csv(colpred+'.csv')
            ypred = model.predict_proba(Xtest)
            yallpred = np.hstack((yallpred, ypred))
            print colpred, (time.time() - t0) / 60.
    return yallpred

In [135]:
def submit(yallpred, colnames, file_name):
    #format
    dresult = pd.DataFrame()
    dresult['id'] = np.arange(len(dtest))
    colorder = argsort(colnames)
    for i in range(104):
        dresult[colnames[colorder[i]]] = yallpred[:, colorder[i]]
    #write
    dresult.to_csv(file_name+'.csv', index = False)
    zf = zipfile.ZipFile(file_name+'.zip', mode='w')
    zf.write(file_name+'.csv', compress_type=zipfile.ZIP_DEFLATED)
    zf.close()
    os.remove(file_name+'.csv')
    print 'submitted!'

In [47]:
%%time
file_name='def_rf_200'
yallpred=rf_predict(rs, 200)
np.savetxt('models/m1_encoded_ts.txt', yallpred)

Function 0.613785632451
Use 0.955710566044
Sharing 1.2887815992
Reporting 1.60017421643
Student_Type 1.95505320231
Position_Type 2.35938735008
Object_Type 2.66021096706
Pre_K 2.96443511645
Operating_Status 3.26638186773
CPU times: user 42min 6s, sys: 1min 9s, total: 43min 16s
Wall time: 3min 34s


/home/atselikov/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [83]:
pd.DataFrame(yallpred).sample(10)

,0,1,2
15438,0,0.000000,1.000000
97665,0,0.000000,1.000000
9285,0,0.000000,1.000000
47290,1,0.000000,0.000000
40560,0,0.000000,1.000000
59021,0,1.000000,0.000000
15699,0,0.000000,1.000000
42580,0,0.054391,0.945609
97754,0,0.000000,1.000000
81580,0,0.000000,1.000000


In [14]:
submit(yallpred, colnames, file_name)

submitted!


In [82]:
sum(finallsd[:,0])

13085

In [66]:
pd.DataFrame(finallsd).describe(include='all')

,0,1,2
count,100000.000000,100000.000000,100000.000000
mean,0.130850,0.031630,0.874970
std,0.337238,0.175014,0.330755
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000
50%,0.000000,0.000000,1.000000
75%,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000


In [100]:
dresult.to_csv(file_name+'.csv', index = False)
zf = zipfile.ZipFile(file_name+'.zip', mode='w')
zf.write(file_name+'.csv', compress_type=zipfile.ZIP_DEFLATED)
zf.close()
os.remove(file_name+'.csv')

def_no_digits.csv
	Comment     : 
	Modified    : 2016-03-15 15:18:08
	System      : Unix
	ZIP version : 20
	Compressed  : 1753632 bytes
	Uncompressed: 42397643 bytes



In [30]:
dall.head(20).to_csv('dall_head20.csv', index = False)

In [29]:
dtrain.head(20).to_csv('dtrain_head20.csv', index = False)
dtest.head(20).to_csv('dtest_head20.csv', index = False)

# META CLASSIFIER

In [97]:
Xall2train = np.hstack((m1train, m2train))
Xall2test = np.hstack((m1test, m2test))
print np.shape(Xall2train)
print np.shape(Xall2test)

(300000, 208)
(100000, 208)


In [98]:
col_set=dall.columns[:9]
rs=10
#folds=3
est_count=200
arres = []
arrlos = []
allypred = np.zeros((len(dtrain), 104))
clf = RandomForestClassifier(n_estimators = est_count, n_jobs = 15, random_state=rs) #0.098330060692
get_kfold_scores(Xall2train, dall[col_set][:len(dtrain)], clf, random_state  = rs)

0.0560509880613
0.0546285653295


KeyboardInterrupt: 